In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [3]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append( {'role': 'system', 'content': self.system} )

    def __call__(self,message=""):
        if message:
            self.messages.append( {'role': 'user', 'content': message} )
        result = self.execute()
        self.messages.append( {'role': 'assistant', 'content': result} )
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

In [4]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

# Tools
def calculate(operation) -> float:
    return eval(operation)

# Can be made to search google
def get_planet_mass(planet) -> float:
    match planet.lower():
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case _:
            return 0.0

## Testing the Agent so far

In [5]:
Rudra = Agent(client=client, system=system_prompt)

In [6]:
result = Rudra("What is the mass of earth times 5?")
print(result)

Thought: To find the mass of Earth times 5, I first need to determine the mass of Earth. 

Action: get_planet_mass: Earth
PAUSE


In [7]:
Rudra.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [8]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [9]:
next_prompt = f"Observation: {observation}"
result = Rudra(next_prompt)
print(result)

Thought: Now that I have the mass of Earth, I can proceed to multiply it by 5 to get the final result.

Action: calculate: 5.972e+24 * 5
PAUSE


In [10]:
Rudra.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [11]:
observation = calculate("3.285e24 * 5")
print(observation)

1.6425000000000001e+25


In [12]:
next_prompt = f"Observation: {observation}"
result = Rudra(next_prompt)
print(result)

Thought: I have successfully calculated the mass of Earth times 5, so now I can provide the answer.

Answer: The mass of Earth times 5 is 1.6425e+25


In [13]:
Rudra.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [22]:
# That regex code
import re
result = """
Thought: I now have the masses of both Earth and Mercury. The next step is to add these two masses together.

Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
"""
action = re.findall( r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE )
print(action)

# we get a list of 1 row with first column giving function, and 2nd column giving parameters.
function = action [0][0]
parameters = action [0][1]

out = f"{function}('{parameters}')"
print(out)
print(eval(out))

[('calculate', '5.972e+24 + 3.285e+23')]
calculate('5.972e+24 + 3.285e+23')
6.300500000000001e+24


In [ ]:
""""
"it makes calculate function call and function call returns the eval() of the string inside. So, the expression inside ''
are executed as a python command and result is returned."

'5.972e+24 + 3.285e+23' is executed as 5.972e+24 + 3.285e+23 and output is returned in float
"""

## Combination

In [14]:
# Loop for the answer
import re

def loop (max_iterations = 3, query = ''):
    agent = Agent(client=client, system=system_prompt)
    tools = ['calculate', 'get_planet_mass']

    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(f'{result}')

        if "PAUSE" in result and "Action" in result:
            action = re.findall( r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE )
            choosen_tool = action[0][0]
            arg = action[0][1]
        
            if choosen_tool in tools:
                result_tool = eval(f"{choosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            print(next_prompt)
            continue
        if "Answer" in result:
            break

loop(max_iterations = 5, query="What is the mass of the earth plus the mass of the mercury and all of it times 5?")

Thought: To find the total mass, I first need to find the mass of Earth and Mercury, then add them together, and finally multiply the result by 5. I'll start by finding the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Mercury to add them together.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I now have the masses of both Earth and Mercury. The next step is to add these two masses together.

Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now that I have the combined mass of Earth and Mercury, I need to multiply this total by 5 to get the final result.

Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have now calculated the total mass of Earth and Mercury multiplied by 5, so I have all the necessary information to provide the final answer.

Answer: The mass of the Eart